양자화 하는 파일

In [1]:
import onnx
import onnxruntime
from onnxruntime.quantization import QuantType, quantize_static, CalibrationDataReader, QuantFormat
from onnxruntime.quantization.shape_inference import quant_pre_process

import funs

import os

import pandas as pd 
import numpy as np  

In [2]:
#### Name Parameters ####

model_fp32_name = 'FRFconv-TDS_onnx'
model_fp32 = f'quant_models/{model_fp32_name}.onnx'

# 전처리된 모델
model_fp32_prep_name = "FRFconv-TDS_onnx_prep"
model_fp32_prep = f'quant_models/{model_fp32_prep_name}.onnx'

# 양자화된 모델
model_quant_name = f'{model_fp32_name}.quant'
model_quant = f'quant_models/{model_quant_name}.onnx'


### 기타 파라미터 ###
config = funs.load_yaml('./config.yaml')

In [3]:
def check_model(model_path):
    onnx.checker.check_model(model_path)
    print(f"{model_path} is valid!")

def preprocess_model(model_path, model_prep_path):
    quant_pre_process(model_path, model_prep_path)
    print(f"Preprocessed model saved to {model_prep_path}")

In [4]:
# 원본 모델 확인
check_model(model_fp32)

# 모델 전처리 및 확인
preprocess_model(model_fp32, model_fp32_prep)
check_model(model_fp32_prep)

quant_models/FRFconv-TDS_onnx.onnx is valid!
Preprocessed model saved to quant_models/FRFconv-TDS_onnx_prep.onnx
quant_models/FRFconv-TDS_onnx_prep.onnx is valid!


In [5]:
# Calibration dataset 제작
funs.set_seed(config.seed)

data_root_dirs = os.path.join(config.dataset_root)

Cylindrical_dirs = funs.get_bearing_paths(data_root_dirs, 'CylindricalRoller', config.rpm, config.sampling_rate)
DepGroove_dirs = funs.get_bearing_paths(data_root_dirs, 'DeepGrooveBall', config.rpm, config.sampling_rate)
Tapered_dirs = funs.get_bearing_paths(data_root_dirs, 'TaperedRoller', config.rpm, config.sampling_rate)

print("Making dataframes...")

Cylindrical_df = funs.make_dataframe(config, Cylindrical_dirs)
DepGroove_df =funs.make_dataframe(config, DepGroove_dirs)
Tapered_df = funs.make_dataframe(config, Tapered_dirs)  

print("concat dataframes...")
all_df = pd.concat([Cylindrical_df, DepGroove_df, Tapered_df], ignore_index=True)

train_df, val_df, test_df = funs.split_dataframe(all_df, 0.6, 0.2)

cali_data, _ = funs.build_from_dataframe(val_df, config.sample_size, config.overlap, False)
cali_data = cali_data.astype(np.float32)

Making dataframes...
concat dataframes...


In [6]:
class MyCalibrationDataReader(CalibrationDataReader):
    def __init__(self, data, model_path):
        self.enum_data = None
        self.data = data 

        # Use inference session to get input shape.
        session = onnxruntime.InferenceSession(model_path, None)
        batch_size, channel, length = session.get_inputs()[0].shape
        self.input_name = session.get_inputs()[0].name
        self.datasize = len(data)

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter([
                {self.input_name: sample[np.newaxis, np.newaxis, :].astype(np.float32)}  # (2048,) → (1, 1, 2048)
                for sample in self.data
            ])
        return next(self.enum_data, None)
    
    def rewind(self):
        self.enum_data = None  # Reset the enumeration of calibration data


In [7]:
dr = MyCalibrationDataReader(cali_data, model_fp32_prep)

In [8]:
quantize_static(
    model_fp32_prep,
    model_quant,
    dr,
    quant_format=QuantFormat.QDQ,
    per_channel=True,
    weight_type=QuantType.QInt8, 
    activation_type=QuantType.QInt8, 
    reduce_range=True,
    extra_options={'WeightSymmetric': True, 'ActivationSymmetric': False}
)

In [9]:
check_model(model_quant)

quant_models/FRFconv-TDS_onnx.quant.onnx is valid!


In [11]:
original_model = onnx.load(model_fp32)
# 양자화 모델  
quant_model = onnx.load(model_quant)

def get_weights_size(model):
    total_size = 0
    for initializer in model.graph.initializer:
        # 실제 가중치 데이터 크기
        total_size += len(initializer.raw_data) if initializer.raw_data else 0
    return total_size

print(f"원본 가중치 크기: {get_weights_size(original_model)} bytes")
print(f"양자화 가중치 크기: {get_weights_size(quant_model)} bytes")

원본 가중치 크기: 35296 bytes
양자화 가중치 크기: 10364 bytes
